In [ ]:
1+1

In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl

from util import *
import var
import cut
import data
import hist

import importlib

from pyanalib import panda_helpers

from scipy import optimize, stats

In [ ]:
plt.rcParams.update({'font.size': 16})
# mpl.rcParams['lines.linewidth'] = 4.

dosave = True
savedir = "plots_2_28_24/signalbox/"

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

In [ ]:
# configured
DETSYST = 9.8711
DETSYST

In [ ]:
NUMISYST = 8.6

In [ ]:
importlib.reload(var)
importlib.reload(cut)
importlib.reload(data)
importlib.reload(hist)
importlib.reload(panda_helpers)

In [ ]:
filedir = "/icarus/data/users/gputnam/DMCP2023G/mc-F-spectra/"

masses = [220, 240, 260, 280, 300, 320, 340]

filetypes = ["F2-Higgs_M%i_nom_evt_spectrum.df" % (M,) for M in masses]
ftype_names = ["$M_S: $ %i MeV" % M for M in masses]

In [ ]:
cvdfs = [pd.read_hdf(filedir + ft) for ft in filetypes]

In [ ]:
cvdfs[0].columns

In [ ]:
# apply the time weight
for i in range(len(cvdfs)):
    cvdfs[i].weight *= cvdfs[i].timeweight
    cvdfs[i].cvweight *= cvdfs[i].timeweight

In [ ]:
when_all = []
for cvdf in cvdfs:
    when_all.append(cvdf.thnumi < 5)

In [ ]:
noms = []
c_corr = []
gc_corr = []
gc_corr_err = []

for cvdf, w in zip(cvdfs, when_all):
    noms.append((cvdf.weight/(cvdf.cathode_weight*cvdf.gap_weight))[w].sum())
    c_corr.append((cvdf.weight/(cvdf.cathode_weight))[w].sum())
    gc_corr.append((cvdf.weight)[w].sum())
    
    N = cvdf.weight[w].sum()
    Nerr = (cvdf.weight[w]*(1+np.sqrt(cvdf.cathode_weight_err**2 + cvdf.gap_weight_err**2))[w]).sum()
    gc_corr_err.append(np.abs(Nerr - N )/N)
    
noms = np.array(noms)
c_corr = np.array(c_corr)
gc_corr = np.array(gc_corr)

In [ ]:
# plt.plot(masses, c_corr/noms, marker=".", linestyle="none", markersize=10, label="Nominal Rate")
plt.errorbar(masses, gc_corr/noms, yerr=gc_corr_err, marker=".", linestyle="none", markersize=10, label="Corrected Rate")
plt.ylabel("Correction / Nominal")
plt.xlabel("Scalar Mass [MeV]")

plt.tight_layout()

if dosave:
    plt.savefig(savedir + "tracksplit_corr_w_unc_scalar.pdf", bbox_inches="tight")
    plt.savefig(savedir + "tracksplit_corr_w_unc_scalar.svg", bbox_inches="tight")

In [ ]:
np.mean(gc_corr_err)*100

In [ ]:
flux_uncs = []
g4_uncs = []
det_uncs = []

for cvdf, w, gerr in zip(cvdfs, when_all, gc_corr_err):
    flux_uncs.append(100*(cvdf[w][["wgt_flux_univ_%i" % i for i in range(100)]].multiply(cvdf.weight[w], 0)).sum().std()/cvdf.weight[w].sum())
    flux_uncs[-1] = np.sqrt(flux_uncs[-1]**2 + NUMISYST**2)
    det_uncs.append(np.sqrt((DETSYST**2 + (100*gerr)**2)))
    
flux_uncs = np.array(flux_uncs)
det_uncs = np.array(det_uncs)
total_uncs = np.sqrt(flux_uncs**2 + det_uncs**2)

In [ ]:
plt.plot(masses, flux_uncs, marker=".", linestyle="none", markersize=10, label="Flux")
plt.plot(masses, det_uncs, marker=".", linestyle="none", markersize=10, label="Detector")

plt.plot(masses, total_uncs, marker=".", linestyle="none", markersize=10, label="Total", color="black")

plt.legend(ncol=4, loc='upper center', bbox_to_anchor=(0.5, 1.15))

plt.xlabel("Mass [MeV]")
plt.ylabel("Uncertainty [%]")

if dosave:
    plt.savefig(savedir + "signalbox_syst_scalar.pdf", bbox_inches="tight")

In [ ]:
det_uncs

In [ ]:
NUMISYST